In [149]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [169]:
# один блок трансформера
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # слой многоголового внимания, который использует входные данные для вычисления взвешенной суммы 
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # feed forward сеть, состоит из 2 полносвязных слоев с фцией активации relu (позволяет модели извлекать более сложные и нелинейные зависимости в данных, дополняя результаты многоголового внимания)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        # слои нормализации (для стабилизации и нормализации выходов блока)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        
        # два дропаут слоя (случайное "выключение" части нейронов во время обучения)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        # проход через многоголовое внимание
        attn_output = self.att(inputs, inputs)
        # дропаутим
        attn_output = self.dropout1(attn_output, training=training)
        # нормализуем
        out1 = self.layernorm1(inputs + attn_output)
        # фидворвард сеть
        ffn_output = self.ffn(out1)
        # еще дропаутим
        ffn_output = self.dropout2(ffn_output, training=training)
        # еще немного нормализуем 
        return self.layernorm2(out1 + ffn_output)

In [170]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        # эмбеддинги для токенов
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        # эмбеддинги для позиции
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    # как слой будет применяться к входным данным во время прямого прохода
    def call(self, x):
        # определяем макс длинну последовательности из x
        maxlen = tf.shape(x)[-1]
        # создаем вектор с позициями (последовательность чисел от 0 до maxlen - 1 с шагом 1)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # применяем этот слой к вектору позиций, чтобы получить эмбеддинги для каждой позиции в последовательности 
        # -> каждая позиция теперь представлена в виде вектора
        positions = self.pos_emb(positions)
        # применяем эмбеддинги слов к входным данным 
        # -> каждая слово в последовательности теперь представлено в виде вектора
        x = self.token_emb(x)
        
        return x + positions

# Data

In [152]:
import importlib
import dataloader

importlib.reload(dataloader)

maxlen = 50
num_words = 20000

# загружаем данные из нашего датасета (уже с учетом токенизации и паддингов)
(x_train, y_train), (x_val, y_val) = dataloader.load_data(num_words=num_words, maxlen = maxlen)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
print(y_train)


7206 Training sequences
7206 Validation sequences
[0. 1. 1. ... 0. 1. 0.]


# Model

In [174]:
embed_dim = 32  # Размерность эмбеддинга для каждого токена
num_heads = 2  # Количество голов внимания
ff_dim = 32  # Размер скрытого слоя в сети прямого распространения внутри трансформера.

# создаем входной слой размера maxlen
inputs = Input(shape=(maxlen,))
# инициализируем слой эмбеддинга, который включает в себя как эмбеддинги слов, так и эмбеддинги позиций.
embedding_layer = TokenAndPositionEmbedding(maxlen, num_words, embed_dim)
# входные данные подаются на слой эмбеддинга, который возвращает эмбеддинги для каждого токена с учетом их позиций.
x = embedding_layer(inputs)
# создаем блок трансформера
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
# эмбеддинги подвергаются обработке блоком трансформера
x = transformer_block(x)
# глобальное усреднение и применение пулинга:
# для уменьшения размерности перед последним слоем классификации. Этот слой выполняет глобальное усреднение значений по всей длине последовательности
x = GlobalAveragePooling1D()(x)
#Применяем dropout и полносвязные слои
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
# создаем выходной слой
outputs = Dense(2, activation="softmax")(x)

# создаем модель
model = Model(inputs=inputs, outputs=outputs)

# Training

In [139]:
# компилируем нейронную сеть
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# обучаем
history = model.fit(x_train, y_train, 
                    batch_size=64, epochs=3, 
                    validation_data=(x_val, y_val)
                   )

Epoch 1/3
181/181 [==============================] - 5s 23ms/step - loss: 0.5779 - accuracy: 0.6988 - val_loss: 0.3807 - val_accuracy: 0.8432
Epoch 2/3
181/181 [==============================] - 4s 23ms/step - loss: 0.2648 - accuracy: 0.8929 - val_loss: 0.4248 - val_accuracy: 0.8193
Epoch 3/3
181/181 [==============================] - 4s 23ms/step - loss: 0.1346 - accuracy: 0.9503 - val_loss: 0.3923 - val_accuracy: 0.8651


# Testing

In [140]:
# Получаем метрики
results = model.evaluate(x_val, y_val, verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

91/91 - 1s - loss: 0.3923 - accuracy: 0.8651 - 513ms/epoch - 6ms/step
loss: 0.392
accuracy: 0.865
